In [2]:
import pandas as pd
import numpy as np

In [3]:
data_df = pd.read_pickle("absorption_mp_data.pkl")
data_df.head()

,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
0,mp-546266,DyBi2IO4,"(Atom('Dy', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0277, 0.0554, 0.0831, 0.1109, 0.1386, ...","[0.0, 0.8362304735329499, 3.309178082718349, 7...","[0.0, 0.0015666666666666665, 0.0031, 0.0046666...","[6.9163, 6.916533333333334, 6.9172, 6.9182, 6....",55.4328,1.3818000000000001
1,mp-9583,K2ZnF4,"(Atom('K', [np.float64(1.2531985199117757), np...","[0.0, 0.0399, 0.0798, 0.1197, 0.1596, 0.1995, ...","[0.0, 0.1362578822994739, 0.4541860898852432, ...","[0.0, 0.0001, 0.00016666666666666666, 0.0002, ...","[2.2021333333333333, 2.202166666666667, 2.2022...",79.7804,4.3866
2,mp-22988,CsGeCl3,"(Atom('Cs', [np.float64(7.78333021211009), np....","[0.0, 0.029, 0.058, 0.087, 0.116, 0.1449, 0.17...","[0.0, 0.515935972729374, 2.0145311056971553, 4...","[0.0, 0.0007, 0.0013666666666666669, 0.0020333...","[3.975733333333333, 3.9758333333333336, 3.9761...",57.9759,2.178
3,mp-861502,AcFeO3,"(Atom('Ac', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0587, 0.1174, 0.176, 0.2347, 0.2934, 0...","[0.0, 11295.053197761259, 44864.495977063896, ...","[0.0, 30.1914, 49.9196, 37.98413333333333, 21....","[67.54320000000001, 59.6215, 29.6067, 1.281, -...",117.3625,0.0
4,mp-1025029,PrHSe,"(Atom('Pr', [np.float64(-2.0265362), np.float6...","[0.0, 0.0365, 0.073, 0.1095, 0.1459, 0.1824, 0...","[0.0, 1.6849892209331498, 6.697792903781345, 1...","[0.0, 0.0026999999999999997, 0.005366666666666...","[8.784633333333334, 8.7851, 8.786566666666667,...",72.9718,1.9455999999999998


In [5]:
# The structure column is already an atom
a_structure = data_df["structure"][1]
print(a_structure)

Atoms(symbols='K2ZnF4', pbc=True, cell=[[3.87855986, 0.0, -1.1469436], [-0.33916743, 3.86370258, -1.1469436], [-0.00357393, -0.00390187, 7.11934679]])


In [ ]:
# What can we do with the whole structure?
print(a_structure.get_all_distances())
print(a_structure.get_cell())
print(a_structure.get_chemical_formula())
print(a_structure.get_atomic_numbers())
print(a_structure.get_chemical_symbols())
print(a_structure.get_distance(a0=0, a1=1))
print(a_structure.get_distances(0,2))
print(a_structure.get_global_number_of_atoms())
print(a_structure.get_initial_charges())
print(a_structure.get_kinetic_energy())
print(a_structure.get_pbc())
print(a_structure.get_positions())
print(a_structure.get_volume())
print(a_structure.get_masses()) # Get atomic mass of each atom in the structure
print(a_structure.get_momenta()) # Get the momenta of each atom in the structure
print(a_structure.get_moments_of_inertia()) # Get the moments of inertia in the structure
print(a_structure.get_angular_momentum())
print(a_structure.get_initial_magnetic_moments())
print(a_structure.get_atomic_numbers())
print(a_structure.get_tags())
print(a_structure.get_angle(0,1,2))

## Get nodes, edges, edge_features, global_attributes, target_attributes

In [ ]:
# Get total number of atoms in the structure
num_atoms = a_structure.get_global_number_of_atoms()

edges = []
edge_features = []

for i in range(num_atoms):
    for j in range(num_atoms):
        if i==j:
            pass
        else:
            atom_pair  = [i,j]
            edges.append(atom_pair)

            dist_pair = [a_structure.get_distance(i,j)]
            edge_features.append(dist_pair)

print(edges)
print(edge_features)

[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [1, 0], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [2, 0], [2, 1], [2, 3], [2, 4], [2, 5], [2, 6], [3, 0], [3, 1], [3, 2], [3, 4], [3, 5], [3, 6], [4, 0], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 6], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5]]
[[np.float64(3.9556341352454414)], [np.float64(4.626179374003504)], [np.float64(5.048881360180091)], [np.float64(5.048881428930239)], [np.float64(2.863020000147007)], [np.float64(2.6003128627683147)], [np.float64(3.9556341352454414)], [np.float64(3.429949303677079)], [np.float64(2.770358140950765)], [np.float64(2.7703578371155673)], [np.float64(2.600312862768315)], [np.float64(2.8630200001470074)], [np.float64(4.626179374003504)], [np.float64(3.429949303677079)], [np.float64(2.022294872736428)], [np.float64(2.022295217930138)], [np.float64(5.326670798770765)], [np.float64(2.0258665112426155)], [np.float64(5.048881360180091)], [np.float64(2.770358140950765

In [ ]:
# Get nodes of structure
all_nodes = []
for i in a_structure:
    the_nodes = [i.index, i.number, i.position]
    all_nodes.append(the_nodes)

print(all_nodes)

[[np.int64(19), 0, array([1.25319852, 1.36802722, 4.23786864])], [np.int64(19), 1, array([2.28261998, 2.49177349, 0.58759095])], [np.int64(30), 2, array([0., 0., 0.])], [np.int64(9), 3, array([ 1.93927993,  0.        , -0.5734718 ])], [np.int64(9), 4, array([-0.16958371,  1.93185129, -0.5734718 ])], [np.int64(9), 5, array([2.98702846, 3.26072587, 2.96963795])], [np.int64(9), 6, array([0.54879004, 0.59907484, 1.85582164])]]


In [66]:
# Get global attributes
global_columns = ["energies", "imag_dielectric", "real_dielectric", "bandgap"]

an_index = 0

global_att = [data_df[a_column][an_index] for a_column in global_columns]
print(global_att)

[array([0.00000e+00, 2.77000e-02, 5.54000e-02, ..., 5.53773e+01,
       5.54050e+01, 5.54328e+01], shape=(2001,)), array([0.        , 0.00156667, 0.0031    , ..., 0.0006    , 0.0006    ,
       0.0006    ], shape=(2001,)), array([6.9163    , 6.91653333, 6.9172    , ..., 0.85943333, 0.85963333,
       0.8598    ], shape=(2001,)), array(1.3818)]


In [74]:
# Get the target values
targ = data_df["absorption_coefficient"][an_index]
print(targ)

[0.00000000e+00 8.36230474e-01 3.30917808e+00 ... 1.81631330e+03
 1.81701042e+03 1.81774592e+03]


## Now as a function that will iterate through the whole dataset

In [ ]:
def get_graphical(row):
    # Get nodes
    all_nodes = []
    for i in row["structure"]:
        the_nodes = [i.index, i.number, i.position]
        all_nodes.append(the_nodes)

    # Get edges and edge features
    num_atoms = row["structure"].get_global_number_of_atoms()

    edges = []
    edge_features = []
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i==j:
                pass
            else:
                atom_pair  = [i,j]
                edges.append(atom_pair)

                dist_pair = [row["structure"].get_distance(i,j)]
                edge_features.append(dist_pair)

    # Get global attributes
    global_columns = ["energies", "imag_dielectric", "real_dielectric", "bandgap"]
    global_att = [row[a_column] for a_column in global_columns]

    # Get target attribute
    target_att = [row["absorption_coefficient"]]

    return all_nodes, edges, edge_features, global_att, target_att

sample = data_df.apply(lambda row: get_graphical(row), axis=1)
print(sample)

In [ ]:
sample[0]

# Turn graph into torch data